In [ ]:
"""
Create region coverage maps - side by side with proper aspect ratio
Shows all visited tiles in peach, with region visited tiles in orange
Shows unvisited tiles in region in grey
Includes lakes, country borders, and water features from OpenStreetMap
Works for any Swiss region (Geneva, Vaud, etc.)
"""

import pandas as pd
import geopandas as gpd
import osmnx as ox
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from shapely.geometry import Point, box

def tile_to_lon_lat(tile_x, tile_y, zoom):
    """Convert tile coordinates to longitude/latitude (center of tile)"""
    n = 2.0 ** zoom
    lon = (tile_x + 0.5) / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * (tile_y + 0.5) / n)))
    lat = math.degrees(lat_rad)
    return lon, lat

def tile_to_bbox(tile_x, tile_y, zoom):
    """Convert tile to bounding box coordinates"""
    n = 2.0 ** zoom
    west = tile_x / n * 360.0 - 180.0
    east = (tile_x + 1) / n * 360.0 - 180.0
    
    lat_rad_north = math.atan(math.sinh(math.pi * (1 - 2 * tile_y / n)))
    north = math.degrees(lat_rad_north)
    
    lat_rad_south = math.atan(math.sinh(math.pi * (1 - 2 * (tile_y + 1) / n)))
    south = math.degrees(lat_rad_south)
    
    return west, east, south, north

def create_region_maps(region, squadrat_file, squadratinho_file, output_dir='output'):
    """
    Create two side-by-side maps showing region coverage at zoom 14 and 17
    
    Parameters:
    -----------
    region : str
        Name of the region (e.g., 'Geneva', 'Vaud', 'Basel')
    squadrat_file : str
        Path to squadrat grid mask CSV file
    squadratinho_file : str
        Path to squadratinho grid mask CSV file
    output_dir : str
        Output directory for saving the maps
    """
    
    print(f"Creating {region} coverage maps...")
    
    # Download region boundary
    print(f"Downloading {region} boundary from OpenStreetMap...")
    if region == "Canton of Geneva":
        region_gdf = ox.geocode_to_gdf(
            {'state': 'Geneva'}, 
            by_osmid=False,
            which_result=1
        )
    else:
        region_gdf = ox.geocode_to_gdf(f'{region}, Switzerland', which_result=1)
    
    region_geom = region_gdf.geometry.iloc[0]
    
    # Get region bounds for map extent
    bounds = region_geom.bounds  # (minx, miny, maxx, maxy)
    lon_min, lat_min, lon_max, lat_max = bounds
    
    # Calculate center latitude for aspect ratio
    center_lat = (lat_min + lat_max) / 2
    
    # Add some padding
    lon_padding = (lon_max - lon_min) * 0.1
    lat_padding = (lat_max - lat_min) * 0.1
    lon_min_padded = lon_min - lon_padding
    lon_max_padded = lon_max + lon_padding
    lat_min_padded = lat_min - lat_padding
    lat_max_padded = lat_max + lat_padding
    
    # Download geographic features from OSM using geometries_from_place
    print("\nDownloading geographic features from OpenStreetMap...")
    
    # Try to get water features for the broader region
    water_gdf = None
    try:
        # Query for water bodies in the Geneva region
        tags_water = {'natural': ['water', 'bay'], 'water': True}
        water_gdf = ox.geometries_from_place('Geneva, Switzerland', tags=tags_water)
        # Clip to our bounding box
        bbox_geom = box(lon_min_padded, lat_min_padded, lon_max_padded, lat_max_padded)
        water_gdf = water_gdf[water_gdf.intersects(bbox_geom)]
        print(f"  Found {len(water_gdf)} water features")
    except Exception as e:
        print(f"  Note: Could not download water features with geometries_from_place: {e}")
        # Try alternative approach with just Geneva Lake
        try:
            water_gdf = ox.geometries_from_place('Lake Geneva', tags={'natural': 'water'})
            print(f"  Found Lake Geneva")
        except:
            print(f"  Could not find Lake Geneva either")
            pass
    
    # Try to get rivers
    rivers_gdf = None
    try:
        tags_rivers = {'waterway': ['river', 'stream']}
        rivers_gdf = ox.geometries_from_place('Geneva, Switzerland', tags=tags_rivers)
        bbox_geom = box(lon_min_padded, lat_min_padded, lon_max_padded, lat_max_padded)
        rivers_gdf = rivers_gdf[rivers_gdf.intersects(bbox_geom)]
        print(f"  Found {len(rivers_gdf)} river features")
    except Exception as e:
        print(f"  Note: Could not download rivers: {e}")
    
    # Try to get administrative boundaries
    admin_gdf = None
    try:
        tags_admin = {'boundary': 'administrative', 'admin_level': ['2', '4']}
        admin_gdf = ox.geometries_from_place('Geneva, Switzerland', tags=tags_admin)
        bbox_geom = box(lon_min_padded, lat_min_padded, lon_max_padded, lat_max_padded)
        admin_gdf = admin_gdf[admin_gdf.intersects(bbox_geom)]
        print(f"  Found {len(admin_gdf)} administrative boundaries")
    except Exception as e:
        print(f"  Note: Could not download admin boundaries: {e}")
    
    # ========== LOAD AND PROCESS SQUADRAT (Zoom 14) ==========
    print("\nProcessing Squadrat (Zoom 14) data...")
    
    df_squadrat = pd.read_csv(squadrat_file)
    
    # Convert tiles to geometry
    tiles_squadrat = []
    for _, row in df_squadrat.iterrows():
        lon, lat = tile_to_lon_lat(row['tile_x'], row['tile_y'], row['zoom'])
        west, east, south, north = tile_to_bbox(row['tile_x'], row['tile_y'], row['zoom'])
        tiles_squadrat.append({
            'lon': lon,
            'lat': lat,
            'west': west,
            'east': east,
            'south': south,
            'north': north,
            'visited': row['inside_polygon'] == 1
        })
    
    # Create GeoDataFrame to check which tiles are in region
    gdf_squadrat = gpd.GeoDataFrame(
        tiles_squadrat,
        geometry=[Point(t['lon'], t['lat']) for t in tiles_squadrat],
        crs='EPSG:4326'
    )
    gdf_squadrat = gdf_squadrat.to_crs(region_gdf.crs)
    gdf_squadrat['in_region'] = gdf_squadrat.within(region_geom)
    gdf_squadrat = gdf_squadrat.to_crs('EPSG:4326')
    
    # Calculate statistics
    tiles_in_region = gdf_squadrat[gdf_squadrat['in_region']]
    total_tiles_region = len(tiles_in_region)
    visited_tiles_region = (tiles_in_region['visited']).sum()
    coverage_squadrat = (visited_tiles_region / total_tiles_region * 100) if total_tiles_region > 0 else 0
    
    print(f"Squadrat - Total in {region}: {total_tiles_region}, Visited: {visited_tiles_region}, Coverage: {coverage_squadrat:.2f}%")
    
    # ========== LOAD AND PROCESS SQUADRATINHO (Zoom 17) ==========
    print("\nProcessing Squadratinho (Zoom 17) data...")
    
    df_squadratinho = pd.read_csv(squadratinho_file)
    
    # Convert tiles to geometry
    tiles_squadratinho = []
    for _, row in df_squadratinho.iterrows():
        lon, lat = tile_to_lon_lat(row['tile_x'], row['tile_y'], row['zoom'])
        west, east, south, north = tile_to_bbox(row['tile_x'], row['tile_y'], row['zoom'])
        tiles_squadratinho.append({
            'lon': lon,
            'lat': lat,
            'west': west,
            'east': east,
            'south': south,
            'north': north,
            'visited': row['inside_polygon'] == 1
        })
    
    # Create GeoDataFrame
    gdf_squadratinho = gpd.GeoDataFrame(
        tiles_squadratinho,
        geometry=[Point(t['lon'], t['lat']) for t in tiles_squadratinho],
        crs='EPSG:4326'
    )
    gdf_squadratinho = gdf_squadratinho.to_crs(region_gdf.crs)
    gdf_squadratinho['in_region'] = gdf_squadratinho.within(region_geom)
    gdf_squadratinho = gdf_squadratinho.to_crs('EPSG:4326')
    
    # Calculate statistics
    tiles_in_region = gdf_squadratinho[gdf_squadratinho['in_region']]
    total_tiles_region = len(tiles_in_region)
    visited_tiles_region = (tiles_in_region['visited']).sum()
    coverage_squadratinho = (visited_tiles_region / total_tiles_region * 100) if total_tiles_region > 0 else 0
    
    print(f"Squadratinho - Total in {region}: {total_tiles_region}, Visited: {visited_tiles_region}, Coverage: {coverage_squadratinho:.2f}%")
    
    # ========== CREATE FIGURE ==========
    print("\nCreating plots...")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    
    for ax in [ax1, ax2]:
        # Plot water features (lakes) FIRST as background
        if water_gdf is not None and len(water_gdf) > 0:
            try:
                # Filter for polygons (lakes)
                water_polygons = water_gdf[water_gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]
                if len(water_polygons) > 0:
                    water_polygons.plot(ax=ax, facecolor='#B3D9FF', edgecolor='#4A90D9', 
                                       linewidth=0.5, alpha=0.6, zorder=1)
            except Exception as e:
                print(f"  Could not plot water: {e}")
        
        # Plot rivers
        if rivers_gdf is not None and len(rivers_gdf) > 0:
            try:
                rivers_gdf.plot(ax=ax, edgecolor='#4A90D9', linewidth=1.0, 
                               alpha=0.5, zorder=1)
            except Exception as e:
                print(f"  Could not plot rivers: {e}")
        
        # Plot administrative borders
        if admin_gdf is not None and len(admin_gdf) > 0:
            try:
                # Filter for country borders (admin_level=2)
                country_borders = admin_gdf[admin_gdf.get('admin_level') == '2']
                if len(country_borders) > 0:
                    for idx, row in country_borders.iterrows():
                        if hasattr(row.geometry, 'boundary'):
                            gpd.GeoSeries([row.geometry]).boundary.plot(
                                ax=ax, edgecolor='#333333', linewidth=1.5, 
                                linestyle='--', alpha=0.8, zorder=2
                            )
            except Exception as e:
                print(f"  Could not plot borders: {e}")
    
    # ========== SQUADRAT PLOT (LEFT) ==========
    ax1.set_title(f'Squadrat Grid (Zoom 14) - {region}', fontsize=16, fontweight='bold', pad=15)
    
    # Plot tiles - Order matters for proper layering
    for idx, row in gdf_squadrat.iterrows():
        west, east, south, north = row['west'], row['east'], row['south'], row['north']
        
        # Determine color and style based on visited and in_region status
        if not row['visited'] and row['in_region']:
            # Unvisited tiles IN REGION - grey
            facecolor = 'lightgrey'
            edgecolor = '#5B4B8A'
            alpha = 0.6
            linewidth = 1.0
            zorder = 3
        elif not row['visited'] and not row['in_region']:
            # Unvisited tiles OUTSIDE REGION - skip to reduce clutter
            continue
        elif row['visited'] and row['in_region']:
            # Visited AND in region - orange
            facecolor = 'tab:orange'
            edgecolor = '#5B4B8A'
            alpha = 0.8
            linewidth = 2.5
            zorder = 5
        else:
            # Visited but NOT in region - peach
            facecolor = '#FFD9B3'
            edgecolor = '#5B4B8A'
            alpha = 0.7
            linewidth = 2.0
            zorder = 4
        
        rect = patches.Rectangle((west, south), east - west, north - south,
                                linewidth=linewidth, edgecolor=edgecolor,
                                facecolor=facecolor, alpha=alpha, zorder=zorder)
        ax1.add_patch(rect)
    
    # Plot region boundary in bold black
    if region_geom.geom_type == 'MultiPolygon':
        for geom in region_geom.geoms:
            region_coords = list(geom.exterior.coords)
            xy = [(lon, lat) for lon, lat in region_coords]
            poly = Polygon(xy, linewidth=3, edgecolor='black', facecolor='none', 
                         alpha=1.0, zorder=10)
            ax1.add_patch(poly)
    else:
        region_coords = list(region_geom.exterior.coords)
        xy = [(lon, lat) for lon, lat in region_coords]
        poly = Polygon(xy, linewidth=3, edgecolor='black', facecolor='none', 
                     alpha=1.0, zorder=10)
        ax1.add_patch(poly)
        
    # Add coverage text
    tiles_in_region_squadrat = gdf_squadrat[gdf_squadrat['in_region']]
    visited_in_region_squadrat = (tiles_in_region_squadrat['visited']).sum()
    ax1.text(0.02, 0.98, f'Coverage: {coverage_squadrat:.2f}%\nTotal tiles in {region}: {len(tiles_in_region_squadrat):,}\nVisited in {region}: {visited_in_region_squadrat:,}',
            transform=ax1.transAxes, fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=11)
    
    ax1.set_xlim([lon_min_padded, lon_max_padded])
    ax1.set_ylim([lat_min_padded, lat_max_padded])
    ax1.set_xlabel('Longitude', fontsize=12)
    ax1.set_ylabel('Latitude', fontsize=12)
    ax1.grid(True, alpha=0.2, linestyle='--', color='gray', zorder=0)
    ax1.set_aspect(1 / np.cos(np.radians(center_lat)))
    
    # ========== SQUADRATINHO PLOT (RIGHT) ==========
    ax2.set_title(f'Squadratinho Grid (Zoom 17) - {region}', fontsize=16, fontweight='bold', pad=15)
    
    # Plot tiles
    for idx, row in gdf_squadratinho.iterrows():
        west, east, south, north = row['west'], row['east'], row['south'], row['north']
        
        # Determine color and style
        if not row['visited'] and row['in_region']:
            # Unvisited tiles IN REGION - grey
            facecolor = 'lightgrey'
            edgecolor = '#5B4B8A'
            alpha = 0.6
            linewidth = 0.5
            zorder = 3
        elif not row['visited'] and not row['in_region']:
            # Unvisited tiles OUTSIDE REGION - skip
            continue
        elif row['visited'] and row['in_region']:
            # Visited AND in region - orange
            facecolor = 'tab:orange'
            edgecolor = '#5B4B8A'
            alpha = 0.8
            linewidth = 1.5
            zorder = 5
        else:
            # Visited but NOT in region - peach
            facecolor = '#FFD9B3'
            edgecolor = '#5B4B8A'
            alpha = 0.7
            linewidth = 1.0
            zorder = 4
        
        rect = patches.Rectangle((west, south), east - west, north - south,
                                linewidth=linewidth, edgecolor=edgecolor,
                                facecolor=facecolor, alpha=alpha, zorder=zorder)
        ax2.add_patch(rect)
    
    # Plot region boundary in bold black
    if region_geom.geom_type == 'MultiPolygon':
        for geom in region_geom.geoms:
            region_coords = list(geom.exterior.coords)
            xy = [(lon, lat) for lon, lat in region_coords]
            poly = Polygon(xy, linewidth=3, edgecolor='black', facecolor='none', 
                         alpha=1.0, zorder=10)
            ax2.add_patch(poly)
    else:
        region_coords = list(region_geom.exterior.coords)
        xy = [(lon, lat) for lon, lat in region_coords]
        poly = Polygon(xy, linewidth=3, edgecolor='black', facecolor='none', 
                     alpha=1.0, zorder=10)
        ax2.add_patch(poly)
    
    # Add coverage text
    tiles_in_region_squadratinho = gdf_squadratinho[gdf_squadratinho['in_region']]
    visited_in_region_squadratinho = (tiles_in_region_squadratinho['visited']).sum()
    ax2.text(0.02, 0.98, f'Coverage: {coverage_squadratinho:.2f}%\nTotal tiles in {region}: {len(tiles_in_region_squadratinho):,}\nVisited in {region}: {visited_in_region_squadratinho:,}',
            transform=ax2.transAxes, fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=11)
    
    ax2.set_xlim([lon_min_padded, lon_max_padded])
    ax2.set_ylim([lat_min_padded, lat_max_padded])
    ax2.set_xlabel('Longitude', fontsize=12)
    ax2.set_ylabel('Latitude', fontsize=12)
    ax2.grid(True, alpha=0.2, linestyle='--', color='gray', zorder=0)
    ax2.set_aspect(1 / np.cos(np.radians(center_lat)))
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/{region}_coverage.png', dpi=150, bbox_inches='tight')
    print(f"\nSaved: {output_dir}/{region}_coverage.png")
    plt.close()
    
    print("\nDone! Map created successfully.")


# Main execution
if __name__ == "__main__":
    
#    squadrat_file = 'output/squadrat_grid_mask_Nyon.csv'
#    squadratinho_file = 'output/squadratinho_grid_mask_Nyon.csv'
    
    squadrat_file = 'output/squadrat_grid_mask_Lausanne.csv'
    squadratinho_file = 'output/squadratinho_grid_mask_Lausanne.csv'

    create_region_maps(
        region='Lausanne',
        squadrat_file=squadrat_file,
        squadratinho_file=squadratinho_file,
        output_dir='figure'
    )


Creating Lausanne coverage maps...

  Note: Could not download water features with geometries_from_place: module 'osmnx' has no attribute 'geometries_from_place'
  Could not find Lake Geneva either
  Note: Could not download rivers: module 'osmnx' has no attribute 'geometries_from_place'
  Note: Could not download admin boundaries: module 'osmnx' has no attribute 'geometries_from_place'

Processing Squadrat (Zoom 14) data...
Squadrat - Total in Lausanne: 0, Visited: 0, Coverage: 0.00%

Processing Squadratinho (Zoom 17) data...
Squadratinho - Total in Lausanne: 0, Visited: 0, Coverage: 0.00%

Creating plots...

Saved: figure/Lausanne_coverage.png

Done! Map created successfully.
